In [ ]:
import networkx as nx
import numpy as np

def density_metrics(G, core_nodes, periphery_nodes):
    """
    밀도 지표를 계산하는 함수
    :param G: 네트워크 그래프
    :type G: networkx.Graph
    :param core_nodes: 코어 노드 리스트
    :type core_nodes: list
    :param periphery_nodes: 주변부 노드 리스트
    :type periphery_nodes: list
    :return: 코어 밀도, 주변부 밀도, 코어-주변부 밀도
    :rtype: dict
    """
    n_core = len(core_nodes)
    n_periphery = len(periphery_nodes)

    E_core = G.subgraph(core_nodes).number_of_edges()
    E_periphery = G.subgraph(periphery_nodes).number_of_edges()
    E_core_periphery = sum(1 for u in core_nodes for v in periphery_nodes if G.has_edge(u, v))

    core_density = (2 * E_core) / (n_core * (n_core - 1)) if n_core > 1 else 0
    periphery_density = (2 * E_periphery) / (n_periphery * (n_periphery - 1)) if n_periphery > 1 else 0
    core_periphery_density = E_core_periphery / (n_core * n_periphery) if n_core > 0 and n_periphery > 0 else 0

    return {
        "core_density": core_density,
        "periphery_density": periphery_density,
        "core_periphery_density": core_periphery_density
    }

def calculate_core_periphery_modularity(G, core_nodes, periphery_nodes, self_loop = False):
    """
    코어-주변부 모듈러리티를 계산하는 함수
    :param G: 네트워크 그래프
    :type G: networkx.Graph
    :param core_nodes: 코어 노드 리스트
    :type core_nodes: list
    :param periphery_nodes: 주변부 노드 리스트
    :type periphery_nodes: list
    :param self_loop: 자기 루프 포함 여부 (기본값: False)
    :type self_loop: bool
    :return: 코어-주변부 모듈러리티 값
    :rtype: float
    """
    # 각 노드에 대한 모듈(코어 또는 주변부) 정보를 담는 딕셔너리 생성
    node_to_module = {node: 1 for node in core_nodes}  # 코어는 1
    node_to_module.update({node: 0 for node in periphery_nodes})  # 주변부는 0
    
    # 그래프의 인접 행렬 가져오기
    A = nx.to_numpy_array(G)
    
    # 각 노드의 차수 계산
    degrees = np.array([deg for _, deg in G.degree()])
    
    # 그래프의 총 에지 수
    m = G.number_of_edges()
    
    # 모듈러리티 초기화
    Q = 0.0
    
    # 모듈러리티 계산
    for i in range(len(G)):
        for j in range(len(G)):
            if not self_loop and i == j:
                continue
            if node_to_module[i] == node_to_module[j]:  # 노드 i와 j가 같은 모듈에 있는지 확인
                if node_to_module[i] == 1:  # 두 노드 모두 코어에 있음
                    Q += A[i, j] - (degrees[i] * degrees[j]) / (2 * m)
                else:  # 두 노드 모두 주변부에 있음
                    Q -= A[i, j] - (degrees[i] * degrees[j]) / (2 * m)
    
    Q = Q / (2 * m)
    
    return Q

def calculate_coreness_centrality(G, core_nodes, periphery_nodes):
    """
    코어-주변부 중심성을 계산하는 함수
    :param G: 네트워크 그래프
    :type G: networkx.Graph
    :param core_nodes: 코어 노드 리스트
    :type core_nodes: list
    :param periphery_nodes: 주변부 노드 리스트
    :type periphery_nodes: list
    :return: 코어-주변부 중심성 딕셔너리
    :rtype: dict
    """
    coreness_centrality = {node: 1 for node in core_nodes}
    coreness_centrality.update({node: 0 for node in periphery_nodes})
    return coreness_centrality

def calculate_mean_core_periphery_score(G, core_nodes, periphery_nodes):
    """
    평균 코어-주변부 점수를 계산하는 함수
    :param G: 네트워크 그래프
    :type G: networkx.Graph
    :param core_nodes: 코어 노드 리스트
    :type core_nodes: list
    :param periphery_nodes: 주변부 노드 리스트
    :type periphery_nodes: list
    :return: 평균 코어-주변부 점수
    :rtype: float
    """
    coreness_centrality = calculate_coreness_centrality(G, core_nodes, periphery_nodes)
    coreness_scores = np.array(list(coreness_centrality.values()))
    return np.mean(coreness_scores)

def attribute_assortativity_core_periphery(G, core_nodes, periphery_nodes):
    """
    코어-주변부 구조의 속성 동류성 계수를 계산하는 함수
    :param G: 네트워크 그래프
    :type G: networkx.Graph
    :param core_nodes: 코어 노드 리스트
    :type core_nodes: list
    :param periphery_nodes: 주변부 노드 리스트
    :type periphery_nodes: list
    :return: 속성 동류성 계수
    :rtype: float
    """
    # 각 노드에 코어-주변부 속성 추가
    for node in core_nodes:
        G.nodes[node]['core_periphery'] = 'core'
    for node in periphery_nodes:
        G.nodes[node]['core_periphery'] = 'periphery'
    
    # 속성 동류성 계수 계산
    assortativity = nx.attribute_assortativity_coefficient(G, 'core_periphery')
    
    return assortativity